Necessary installs


In [3]:
# %pip install transformers
# %pip install torch transformers
%pip install hf_xet

   ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/4.1 MB 5.6 MB/s eta 0:00:01
   -------------------- ------------------- 2.1/4.1 MB 4.9 MB/s eta 0:00:01
   --------------------------------- ------ 3.4/4.1 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 4.1/4.1 MB 5.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Draft 1

In [6]:
# Required Libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load a pre-trained LLM
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Simulate basic memory (e.g., what happened previously)
game_memory = []

# Difficulty state (0: easy, 1: medium, 2: hard)
difficulty = 1

# Function to generate story continuation
def generate_story(context, player_input, difficulty):
    prompt = f"Continue the fantasy adventure. Difficulty: {['easy','medium','hard'][difficulty]}. "
    prompt += f"Story so far: {context} Player says: {player_input}"
    
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
    output = model.generate(**inputs, max_length=150)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Simulate difficulty adjustment (stub for RL)
def adjust_difficulty(player_input):
    global difficulty
    # Example logic: if input contains "attack" or "fight", increase difficulty
    if "attack" in player_input.lower() or "fight" in player_input.lower():
        difficulty = min(difficulty + 1, 2)
    elif "talk" in player_input.lower() or "run" in player_input.lower():
        difficulty = max(difficulty - 1, 0)
    # In a real system, an RL agent would optimize this
    return difficulty

# Start of the game
context = "You awaken in a dark forest. A mysterious figure approaches you."

print("Welcome to the AI Dungeon!")
while True:
    print("\nStory so far:")
    print(context)
    
    player_input = input("What do you do? (type 'quit' to exit): ")
    if player_input.lower() == 'quit':
        break
    
    game_memory.append(player_input)
    difficulty = adjust_difficulty(player_input)
    
    context = generate_story(context, player_input, difficulty)


Using device: cpu
Welcome to the AI Dungeon!

Story so far:
You awaken in a dark forest. A mysterious figure approaches you.

Story so far:
fantasy adventure. Difficulty: medium. Story so far: You awaken in a dark forest. A mysterious figure approaches you. Player says: hello


Draft 2

In [8]:
# Required Libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load a pre-trained LLM
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Simulate basic memory (e.g., what happened previously)
game_memory = []

# Difficulty state (0: easy, 1: medium, 2: hard)
difficulty = 1

# Function to generate story continuation
def generate_story(context, player_input, difficulty):
    difficulty_level = ['easy', 'medium', 'hard'][difficulty]
    
    prompt = (
        f"Continue the fantasy adventure. Difficulty: {difficulty_level}. "
        f"Story so far: {context} "
        f"Player action: {player_input}"
    )
    
    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
    output = model.generate(**inputs, max_length=150)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return response

# Simulate difficulty adjustment (stub for RL)
def adjust_difficulty(player_input):
    global difficulty
    if "attack" in player_input.lower() or "fight" in player_input.lower():
        difficulty = min(difficulty + 1, 2)
    elif "talk" in player_input.lower() or "run" in player_input.lower():
        difficulty = max(difficulty - 1, 0)
    return difficulty

# Start of the game
initial_context = "You awaken in a dark forest. A mysterious figure approaches you."
game_memory.append(initial_context)
context = initial_context

print("🌲 Welcome to the AI Dungeon! 🌲")
while True:
    print("\n📖 Story so far:")
    print(context)
    
    player_input = input("\n💬 What do you do? (type 'quit' to exit): ")
    if player_input.lower() == 'quit':
        print("🛑 Game ended. Thanks for playing!")
        break
    
    # Adjust difficulty
    difficulty = adjust_difficulty(player_input)

    # Update memory
    game_memory.append(f"Player: {player_input}")
    
    # Create a rolling context window (last 5 items)
    recent_context = " ".join(game_memory[-5:])
    
    # Generate new part of the story
    new_story = generate_story(recent_context, player_input, difficulty)
    game_memory.append(f"AI: {new_story}")
    
    # Print the generated continuation
    print("\n🧙 AI continues the story:")
    print(new_story)

    # Update context
    context = new_story

    # Save to log file
    with open("story_log.txt", "a") as log:
        log.write(f"Player: {player_input}\n")
        log.write(f"AI: {new_story}\n\n")


Using device: cpu
🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
You awaken in a dark forest. A mysterious figure approaches you.

🧙 AI continues the story:
fantasy adventure. Difficulty: medium. Story so far: You awaken in a dark forest. A mysterious figure approaches you. Player action: I say hello to the mysterious figure. Player action: I say hello to the mysterious figure.

📖 Story so far:
fantasy adventure. Difficulty: medium. Story so far: You awaken in a dark forest. A mysterious figure approaches you. Player action: I say hello to the mysterious figure. Player action: I say hello to the mysterious figure.

🧙 AI continues the story:
: medium. Story so far: You awaken in a dark forest. A mysterious figure approaches you. Player action: hello

📖 Story so far:
: medium. Story so far: You awaken in a dark forest. A mysterious figure approaches you. Player action: hello
🛑 Game ended. Thanks for playing!


Draft 3

In [1]:
# Required Libraries
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load a pre-trained LLM
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Simulate basic memory
game_memory = []

# Difficulty state (0: easy, 1: medium, 2: hard)
difficulty = 1

# Function to generate story continuation
def generate_story(context, player_input, difficulty):
    difficulty_level = ['easy', 'medium', 'hard'][difficulty]
    
    # Use a structured prompt format T5 understands
    prompt = (
        f"Continue this fantasy story at {difficulty_level} difficulty:\n"
        f"{context}\n"
        f"Player: {player_input}\n"
        f"Story:"
    )

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
    output = model.generate(
        **inputs,
        max_length=150,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Simulate difficulty adjustment
def adjust_difficulty(player_input):
    global difficulty
    if "attack" in player_input.lower() or "fight" in player_input.lower():
        difficulty = min(difficulty + 1, 2)
    elif "talk" in player_input.lower() or "run" in player_input.lower():
        difficulty = max(difficulty - 1, 0)
    return difficulty

# Start of the game
initial_context = "You awaken in a dark forest. A mysterious figure approaches you."
game_memory.append(initial_context)
context = initial_context

print("🌲 Welcome to the AI Dungeon! 🌲")

while True:
    print("\n📖 Story so far:")
    print(context)

    player_input = input("\n💬 What do you do? (type 'quit' to exit): ")
    if player_input.lower() == 'quit':
        print("🛑 Game ended. Thanks for playing!")
        break

    # Adjust difficulty
    difficulty = adjust_difficulty(player_input)

    # Generate the next part of the story
    new_story = generate_story(context, player_input, difficulty)

    # Print and update context
    print("\n🧙 AI continues the story:")
    print(new_story)

    game_memory.append(f"Player: {player_input}")
    game_memory.append(f"{new_story}")

    # Only keep recent memory to avoid prompt overflow
    context = " ".join(game_memory[-6:])

    # Save to log file
    with open("story_log.txt", "a") as log:
        log.write(f"Player: {player_input}\n")
        log.write(f"AI: {new_story}\n\n")



c:\Ihno\Hoger onderwijs\UCLL\Jaar 2\Advanced AI\semester project\avanced_ai_semester_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
You awaken in a dark forest. A mysterious figure approaches you.

🧙 AI continues the story:
fantasy story at easy difficulty: You awaken in a dark forest. A mysterious figure approaches you. Player: I try to run from the mystery figure. Story:

📖 Story so far:
You awaken in a dark forest. A mysterious figure approaches you. Player: I try to run from the mysterious figure. fantasy story at easy difficulty: You awaken in a dark forest. A mysterious figure approaches you. Player: I try to run from the mystery figure. Story:

🧙 AI continues the story:
this fantasy story at easy difficulty: You awaken in a dark forest. A mysterious figure approaches you. Player: I try to run from the mystery figure.

📖 Story so far:
You awaken in a dark forest. A mysterious figure approaches you. Player: I try to run from the mysterious figure. fantasy story at easy difficulty: You awaken in a dark forest. A mysterious figure approaches you. 

Draft 4


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json
import os

# Load model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Character setup
player_name = input("Enter your character's name: ")
inventory = ["torch"]
stats = {"health": 100, "strength": 10, "gold": 5}
difficulty = 1  # 0: Easy, 1: Medium, 2: Hard

# Load game state if it exists
save_file = "savegame.json"
if os.path.exists(save_file):
    with open(save_file, "r") as f:
        save = json.load(f)
        game_memory = save["game_memory"]
        inventory = save["inventory"]
        stats = save["stats"]
        difficulty = save["difficulty"]
        print("✅ Game loaded.")
else:
    game_memory = [f"{player_name} awakens in a dark forest. A mysterious figure approaches."]

# Generate story continuation
def generate_story(context_snippet):
    prompt = f"Continue the story:\n{context_snippet}\nNext:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    output = model.generate(
        **inputs,
        max_length=150,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Adjust difficulty
def adjust_difficulty(player_input):
    global difficulty
    if "attack" in player_input.lower():
        difficulty = min(difficulty + 1, 2)
        stats["strength"] += 1
    elif "run" in player_input.lower():
        difficulty = max(difficulty - 1, 0)
    return difficulty

# Save game state
def save_game():
    with open(save_file, "w") as f:
        json.dump({
            "game_memory": game_memory,
            "inventory": inventory,
            "stats": stats,
            "difficulty": difficulty
        }, f, indent=2)
    print("💾 Game saved.")

# Game loop
print("\n🌲 Welcome to the AI Dungeon! 🌲")

while True:
    print("\n📖 Story so far:")
    print(game_memory[-1])
    print(f"\n🧍 {player_name}'s Inventory: {inventory}")
    print(f"❤️ Stats: {stats} | 🎯 Difficulty: {['Easy','Medium','Hard'][difficulty]}")

    player_input = input("\n💬 What do you do? (type 'save', 'load', or 'quit'): ")

    if player_input.lower() == 'quit':
        print("🛑 Game ended. Thanks for playing!")
        break
    elif player_input.lower() == 'save':
        save_game()
        continue
    elif player_input.lower() == 'load':
        if os.path.exists(save_file):
            with open(save_file, "r") as f:
                save = json.load(f)
                game_memory = save["game_memory"]
                inventory = save["inventory"]
                stats = save["stats"]
                difficulty = save["difficulty"]
            print("✅ Game loaded.")
        else:
            print("⚠️ No saved game found.")
        continue

    adjust_difficulty(player_input)

    # Build a small context window for story continuity
    context_snippet = "\n".join(game_memory[-3:] + [f"Player: {player_input}"])

    # Generate new story content
    new_story = generate_story(context_snippet)

    # Update game memory
    game_memory.append(f"Player: {player_input}")
    game_memory.append(new_story)

    # Save logs
    with open("story_log.txt", "a") as log:
        log.write(f"Player: {player_input}\n")
        log.write(f"AI: {new_story}\n\n")


c:\Ihno\Hoger onderwijs\UCLL\Jaar 2\Advanced AI\semester project\avanced_ai_semester_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu

🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium
💾 Game saved.

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium
✅ Game loaded.

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium
✅ Game loaded.

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium
✅ Game loaded.

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'stren

Draft 5

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import random
import os
import json

# Load model/tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# model_name = "google/flan-t5-base"  # More capable than t5-small
model_name = "t5-small"  # Loads quickly, good for dev work
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Game state variables
player_name = "Ihno"
inventory = ["torch"]
stats = {"health": 100, "strength": 10, "gold": 5}
difficulty = 1  # 0: easy, 1: medium, 2: hard
game_memory = [f"{player_name} awakens in a dark forest. A mysterious figure approaches."]
save_file = "game_state.json"

# Generate story continuation
def generate_story(context, player_input, difficulty):
    difficulty_level = ['easy', 'medium', 'hard'][difficulty]

    prompt = (
        f"You are a narrator guiding a fantasy story at {difficulty_level} difficulty.\n"
        f"The hero, {player_name}, is on an adventure.\n"
        f"Current situation:\n{context}\n"
        f"{player_name} says or does: \"{player_input}\"\n"
        f"Continue the story based on this."
    )

    inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)
    output = model.generate(
        **inputs,
        max_length=150,
        temperature=0.9,        # Adds creativity
        top_p=0.95,             # Nucleus sampling
        do_sample=True,         # Enables sampling
        num_return_sequences=1  # One continuation
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response.strip()


# Difficulty adjustment based on keywords
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "charge", "punch"]):
        difficulty = min(difficulty + 1, 2)
    elif any(word in player_input.lower() for word in ["talk", "negotiate", "run", "hide"]):
        difficulty = max(difficulty - 1, 0)

# Save game state
def save_game():
    with open(save_file, "w") as f:
        json.dump({
            "player_name": player_name,
            "inventory": inventory,
            "stats": stats,
            "difficulty": difficulty,
            "game_memory": game_memory
        }, f)
    print("💾 Game saved.")

# Load game state
def load_game():
    global inventory, stats, difficulty, game_memory
    if os.path.exists(save_file):
        with open(save_file, "r") as f:
            save = json.load(f)
            inventory = save["inventory"]
            stats = save["stats"]
            difficulty = save["difficulty"]
            game_memory = save["game_memory"]
        print("✅ Game loaded.")
    else:
        print("⚠️ No saved game found.")

# Game loop
print("\n🌲 Welcome to the AI Dungeon! 🌲")

while True:
    print("\n📖 Story so far:")
    print("\n".join(game_memory[-2:]))

    print(f"\n🧍 {player_name}'s Inventory: {inventory}")
    print(f"❤️ Stats: {stats} | 🎯 Difficulty: {['Easy','Medium','Hard'][difficulty]}")

    player_input = input("\n💬 What do you do? (type 'save', 'load', or 'quit'): ").strip().lower()

    if player_input == "quit":
        print("🛑 Game ended. Thanks for playing!")
        break
    elif player_input == "save":
        save_game()
        continue
    elif player_input == "load":
        load_game()
        continue

    # Update difficulty and add player action
    adjust_difficulty(player_input)
    game_memory.append(f"{player_name}: {player_input}")
    game_memory.append(f"AI: {new_story}")


    # Get short context and generate response
    context_snippet = "\n".join(game_memory[-6:])
    new_story = generate_story(context_snippet, player_input, difficulty)

    # Log everything
    with open("story_log.txt", "a", encoding="utf-8") as log:
        log.write(f"Player: {player_input}\n")
        log.write(f"AI: {new_story}\n\n")
        log.flush()

    # Avoid duplication but still move the story forward
    if new_story and new_story not in game_memory:
        game_memory.append(new_story)
    else:
        fallback = "The story pauses momentarily... perhaps something is about to happen."
        game_memory.append(fallback)


Using device: cpu

🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium

📖 Story so far:
AI: : Ihno awakens in a dark forest. A mysterious figure approaches. Ihno: i talk wit the mysterious figure. this fantasy story at easy difficulty: Ihno awakens in a dark forest. A mysterious figure approaches. Ihno: what does the stranger talk to me about Story:
: the strange figure approaches. A mysterious figure approaches. Ihno: i talk wit the mysterious figure. this fantasy story at easy difficulty: Ihno awakens in a dark forest. A mysterious figure approaches. ihno: what does the stranger talk to me about Story: Ihno says or does: "i talk wit the mysterious figure"

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Easy
⚠️ No saved game found.

📖 Story so far:
AI: : Ihno awakens in 

Draft 7

In [14]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Model and device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "google/flan-t5-base"
AutoTokenizer.from_pretrained(model_name)
AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# Game variables
player_name = "Ihno"
save_file = "game_state.json"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# Game functions
def generate_story(context, player_input, difficulty):
    prompt = (
        f"Continue this fantasy story at {['Easy', 'Medium', 'Hard'][difficulty]} difficulty:\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        f"Story:"
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = model.generate(**inputs, max_length=150, num_return_sequences=1, do_sample=True, temperature=0.9)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 2)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 0)
    return difficulty

def save_game():
    with open(save_file, "w", encoding="utf-8") as f:
        json.dump({
            "context": context,
            "stats": player_stats,
            "inventory": inventory,
            "difficulty": difficulty,
            "memory": game_memory
        }, f, indent=4)

def delete_save():
    if os.path.exists(save_file):
        os.remove(save_file)
        print("🗑️ Previous save data deleted.")
    else:
        print("⚠️ No save file found to delete.")

# Start menu
game_running = False
while not game_running:
    choice = input("\n💾 Load game (l), 🆕 New game (n), 🗑️ Delete save (d), or ❌ Quit (q)? ").strip().lower()
    if choice == 'l':
        if os.path.exists(save_file):
            with open(save_file, "r", encoding="utf-8") as f:
                game_state = json.load(f)
                context = game_state["context"]
                player_stats = game_state["stats"]
                inventory = game_state["inventory"]
                difficulty = game_state["difficulty"]
                game_memory = game_state["memory"]
            print("✅ Game loaded.")
            game_running = True
        else:
            print("⚠️ No saved game found.")
    elif choice == 'n':
        context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
        game_memory = [context]
        player_stats = {"health": 100, "strength": 10, "gold": 5}
        inventory = ["torch"]
        difficulty = 1
        print("🆕 New game started.")
        game_running = True
    elif choice == 'd':
        delete_save()
    elif choice == 'q':
        print("🛑 Exiting game.")
        exit()
    else:
        print("❓ Invalid choice.")

# --- GAME LOOP ---
print("\n🌲 Welcome to the AI Dungeon! 🌲")
while True:
    print("\n📖 Story so far:")
    print(context)
    print(f"\n🧍 {player_name}'s Inventory: {inventory}")
    print(f"❤️ Stats: {player_stats} | 🎯 Difficulty: {['Easy', 'Medium', 'Hard'][difficulty]}")
    
    player_input = input("\n💬 What do you do? (type 'save', 'quit' or your action): ").strip()
    
    if player_input.lower() == 'quit':
        print("🛑 Game ended. Thanks for playing!")
        break
    elif player_input.lower() == 'save':
        save_game()
        print("💾 Game saved.")
        continue

    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    # Get latest context (e.g. last 6 lines)
    context_snippet = "\n".join(game_memory[-6:])
    new_part = generate_story(context_snippet, player_input, difficulty)
    
    context += f"\n{player_name}: {player_input}\n{new_part}"
    game_memory.append(new_part)

    # Log everything
    with open("story_log.txt", "a", encoding="utf-8") as log:
        log.write(f"{player_name}: {player_input}\n{new_part}\n\n")



🆕 New game started.

🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.
Ihno: hello?
Ihno wakes up in the dark forest. He watches his brothers play with the same sword.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.
Ihno: hello?
Ihno wakes up in the dark forest. He watches his brothers play with the same sword.
Ihno: 
Ihno tells the boy to bring up his brother.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium
🛑 Game ended. Thanks for playing!
